In [1]:
import torch
# leaf variable 就是直接create的Tensor，他们不是其他操作的结果
x = torch.ones(2,2, requires_grad=True)
# 像x.t_() 这种in-place operation不能在requires_grad=True中使用
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [2]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [3]:
# 所谓的grad_fn 就是创造这个Tensor的function
print(y.grad_fn)

In [4]:
print(y)
print(y.t_())

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<TBackward>)


In [5]:
# 和TF一样，这里的相乘都是elementwise
# 假如这里将表达式改成z = y * y.t_() * 3 ，这里不会得到matrix相乘的结果，因为Tensor.t_()是一个in-place operation
# 所以在evaluate这个式子数值的时候也会将y改变，在这个expression中y与y.t_()是等效。
z = y * y * 3
# Tensor.mean() 等效于torch.mean()
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [6]:
# 不同于torch.rand()，这里生成的是从N(0,1) Gaussian Distribution采样的Tensor
a = torch.randn(2,2)
a = ((a * 3) / (a - 1))
# requires_grad 这个flag by default 是 false的
print(a.requires_grad)
# Tensor.requires_grad_()能将Tensor.requires_grad这个flag改变，如True，就开始追踪，有grad_fn
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b)
print(b.grad_fn)

False
True
tensor(4.1259, grad_fn=<SumBackward0>)


In [7]:
# Tensor.backward() 开始backpropagation
out.backward()

In [11]:
# 求出out对x的导数，因为x为一个矩阵，求出来的导数跟x形状一致
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


<img src="./fig/backpropagation.png" alt="图片替换文本" width="500" height="713" align="left" />

In [15]:
x = torch.randn(3, requires_grad=True)
# Tensor.data.norm()等于torch.norm(Tensor)，在默认情况下计算的是Forbenius Norm(matrix) 或者 L2 Norm(Vector)
y = x * 2
while y.data.norm() < 1000:
    print(y.data.norm(), y)
    y = y * 2
    
print(y)

tensor(2.5964) tensor([ 2.4776, -0.6099, -0.4804], grad_fn=<MulBackward0>)
tensor(5.1929) tensor([ 4.9553, -1.2197, -0.9608], grad_fn=<MulBackward0>)
tensor(10.3857) tensor([ 9.9106, -2.4394, -1.9216], grad_fn=<MulBackward0>)
tensor(20.7714) tensor([19.8212, -4.8788, -3.8431], grad_fn=<MulBackward0>)
tensor(41.5428) tensor([39.6424, -9.7577, -7.6863], grad_fn=<MulBackward0>)
tensor(83.0857) tensor([ 79.2847, -19.5154, -15.3726], grad_fn=<MulBackward0>)
tensor(166.1714) tensor([158.5694, -39.0308, -30.7451], grad_fn=<MulBackward0>)
tensor(332.3428) tensor([317.1389, -78.0615, -61.4903], grad_fn=<MulBackward0>)
tensor(664.6855) tensor([ 634.2777, -156.1231, -122.9805], grad_fn=<MulBackward0>)
tensor([1268.5554, -312.2461, -245.9610], grad_fn=<MulBackward0>)


In [16]:
# 这里y是一个向量， x也是一个向量，dy/dx应该是一个Jacobian矩阵， 但是若果y与x是更高维的Tensor，这里就无法给出导数的定义，
# 所以PyTorch规定，Tensor.backward()，若果backward中没有额外的参数就不能求导，只能求scalar对Tensor的导数
# 这里给出一个简单的例子，假设y也是一个中间变量，已知最终结果对y的求导结果为v，求最终结果对于x的求导
# 可以理解为(y_transpose * v)得到一个scalar后再到x求导，这里的v可以像上面一个理解为最终结果对y的求导，
# 也可以理解为对y各个分量求和的权重
v = torch.tensor([0.1, 1.0, 0.001], dtype=torch.float)
y.backward(v)
print(x.grad)

tensor([ 102.4000, 1024.0000,    1.0240])


In [18]:
# x在上一个cell开了requires_grad，所以在它基础计算的Tensor自然也都是开了追踪
# 除非有no_grad()暂时把追踪关闭
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
    print((x**2).requires_grad)

True
True
False


In [19]:
# 或者使用Tensor.detach()，返回的值与x一样但是不再具有追踪功能
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
# Tensor0.eq(Tensor1) 与torch.eq()一样，会返回一个elementwise的BoolTensor
# BoolTensor.all()会返回另一个boolTensor，如果BoolTensor全部都是True，它也返回一个True,否则只要有一个是False就返回一个False
print(x.eq(y).all())

True
False
tensor(True)
